In [ ]:
import os
import numpy as np
import pandas as pd
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

df = pd.read_csv('https://raw.githubusercontent.com/Neha-Chiluka/deeplearning/refs/heads/main/tensorflow/data/winequalityN.csv')
df.sample(5)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
5885,red,7.1,0.360,0.30,1.60,0.080,35.0,70.0,0.99693,3.44,0.50,9.4,5
1022,white,6.8,0.190,0.32,7.05,0.019,54.0,188.0,0.99350,3.25,0.37,11.1,8
2352,white,6.1,0.280,0.16,1.30,0.060,36.0,126.0,0.99353,3.13,0.46,8.7,6
5947,red,9.3,0.500,0.36,1.80,0.084,6.0,17.0,0.99704,3.27,0.77,10.8,6
6076,red,5.6,0.915,0.00,2.10,0.041,17.0,78.0,0.99346,3.68,0.73,11.4,5


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Prepare the data
df = df.dropna()
df['is_white_wine'] = [1 if typ == 'white' else 0 for typ in df['type']]
df['is_good_wine'] = [1 if quality >= 6 else 0 for quality in df['quality']]
df.drop(['type', 'quality'], axis=1, inplace=True)

# Train/test split
X = df.drop('is_good_wine', axis=1)
y = df['is_good_wine']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import tensorflow as tf

cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/model-{epoch:02d}-{val_accuracy:.2f}.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
cb_reducelr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    factor=0.1,
    patience=10,
    verbose=1,
    min_lr=0.00001
)

In [ ]:
cb_earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    min_delta=0.001,
    patience=10,
    verbose=1
)

In [ ]:
cb_csvlogger = tf.keras.callbacks.CSVLogger(
    filename='training_log.csv',
    separator=',',
    append=False
)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

model.fit(
    X_train_scaled,
    y_train,
    epochs=1000,
    validation_data=(X_test_scaled, y_test),
    callbacks=[cb_checkpoint, cb_reducelr, cb_earlystop, cb_csvlogger]
)

Epoch 1/1000
145/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6715 - loss: 0.5980
Epoch 1: val_accuracy improved from -inf to 0.75483, saving model to checkpoints/model-01-0.75.keras
162/162 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6770 - loss: 0.5925 - val_accuracy: 0.7548 - val_loss: 0.4918 - learning_rate: 0.0010
Epoch 2/1000
146/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7733 - loss: 0.4913
Epoch 2: val_accuracy improved from 0.75483 to 0.75870, saving model to checkpoints/model-02-0.76.keras
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7721 - loss: 0.4920 - val_accuracy: 0.7587 - val_loss: 0.4816 - learning_rate: 0.0010
Epoch 3/1000
150/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7722 - loss: 0.4783
Epoch 3: val_accuracy did not improve from 0.75870
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7717 - loss: 0.4788 - val_accuracy: 0.7572 - val_loss: 0.4819 - learning_rate: 0.0010
Epoch 4/1000
160/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - a

In [ ]:
from google.colab import files
files.download('training_log.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>